In [1]:
# Load all packages necessary for analysis
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import timeit
import filecmp
import url_stripper

# deck_urls

## Gather

In [2]:
# get a list of file names in type_html_files folder and name it dir_names
dir_names = []
cur_dir = 'C:/Users/muroc/Documents/MTG/type_html_files/'

with os.scandir(cur_dir) as folder:
    for file in folder: 
        if file.is_file():
            dir_names.append(file.name)

In [3]:
# get a list of all urls in all files
deck_urls = []
for i in np.arange(np.count_nonzero(dir_names)):
    with open('C:/Users/muroc/Documents/MTG/type_html_files/' + dir_names[i]) as file:
        soup = BeautifulSoup(file, 'lxml')
    urls = soup.find_all(href=re.compile(r'event\?e\=.+\&d\=.+\&f\=ST'))
    for a in urls:
        deck_urls.append(a)

In [4]:
# get a list of all player names in all files
player_names = []
for i in np.arange(np.count_nonzero(dir_names)):
    with open('C:/Users/muroc/Documents/MTG/type_html_files/' + dir_names[i]) as file:
        soup = BeautifulSoup(file, 'lxml')
    urls = soup.find_all(href=re.compile(r'search\?player'))
    for a in urls:
        player_names.append(a)

In [5]:
# turn deck urls into a dataframe
deck_urls = {'url': deck_urls}
deck_urls = pd.DataFrame(data=deck_urls)

In [6]:
# get deck names and clean them up
url_stripper.url_stripper(deck_urls, 'url', 'name', '(>.+<)')

In [7]:
# extract url from the url column
deck_urls['url'] = deck_urls.url.str.extract('(event.+f\=ST)')

In [10]:
# add root url to the data
urls = []
names = []

for i in np.arange(deck_urls.shape[0]):
    urls.append('https://www.mtgtop8.com/' + str(deck_urls.url[i]))
    names.append(str(deck_urls.name[i]) + '.html')

In [11]:
# clean urls and name
deck_urls['url'] = urls
deck_urls['name'] = names
deck_urls['name'] = deck_urls.name.str.replace(' ', '_')
deck_urls['name'] = deck_urls.name.str.replace('/', '-')

In [12]:
deck_urls.to_csv('C:/Users/muroc/Documents/MTG/data/deck_urls.csv')